# Module 4: Combining Many Weak Learners

**Goal:** Understand gradient boosting (LightGBM/XGBoost) and tune for real performance

**Time:** ~20 minutes

**What you'll do:**
1. Train LightGBM and compare to random forest
2. Understand the key hyperparameters
3. Detect and prevent overfitting
4. Use early stopping

---

## Setup

In [ ]:
# Install lightgbm if needed (Colab has it pre-installed)
try:
    import lightgbm as lgb
except ImportError:
    !pip install lightgbm -q
    import lightgbm as lgb

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Load data
try:
    df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/streamcart_customers.csv')
except:
    df = pd.read_csv('../data/streamcart_customers.csv')

# Prepare features
features = ['tenure_months', 'logins_last_7d', 'logins_last_30d',
            'support_tickets_last_30d', 'items_skipped_last_3_boxes', 'nps_score']

X = df[features].fillna(-1)  # LightGBM handles -1 as missing
y = df['churn_30d']

# Split: train, validation, test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"Train: {len(X_train):,} | Validation: {len(X_val):,} | Test: {len(X_test):,}")

---

## Part 1: Train LightGBM

LightGBM = fast gradient boosting. Industry standard for tabular data.

In [ ]:
# Basic LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)

lgb_model.fit(X_train, y_train)

# Evaluate
lgb_probs = lgb_model.predict_proba(X_test)[:, 1]
lgb_auc = roc_auc_score(y_test, lgb_probs)

print(f"LightGBM AUC: {lgb_auc:.3f}")

In [ ]:
# Compare to random forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)
rf_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])

print(f"=== AUC Comparison ===")
print(f"Random Forest: {rf_auc:.3f}")
print(f"LightGBM:      {lgb_auc:.3f}")
print(f"\nDifference: {lgb_auc - rf_auc:+.3f}")

---

## Part 2: The Key Hyperparameters

These 4 parameters matter most. Learn what they do.

In [ ]:
# The Big 4 hyperparameters
hyperparams = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  HYPERPARAMETER    │  WHAT IT DOES                │  TYPICAL RANGE         │
├─────────────────────────────────────────────────────────────────────────────┤
│  n_estimators      │  Number of trees (rounds)    │  100-1000              │
│  max_depth         │  How deep each tree goes     │  3-10                  │
│  learning_rate     │  Step size per tree          │  0.01-0.3              │
│  num_leaves        │  Max leaves per tree         │  15-127                │
└─────────────────────────────────────────────────────────────────────────────┘

KEY INSIGHT: learning_rate and n_estimators trade off.
- Lower learning_rate + more trees = same result but slower
- Lower learning_rate usually gives better generalization
"""
print(hyperparams)

In [ ]:
# TODO: Experiment with different learning rates
#
# Train 3 models with learning_rate = [0.01, 0.1, 0.3]
# All with n_estimators=100, max_depth=5

learning_rates = [0.01, 0.1, 0.3]
results = []

for lr in learning_rates:
    model = lgb.LGBMClassifier(
        n_estimators=100, max_depth=5, learning_rate=lr, random_state=42, verbose=-1
    )
    model.fit(X_train, y_train)
    
    train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    results.append({'learning_rate': lr, 'train_auc': train_auc, 'test_auc': test_auc})

results_df = pd.DataFrame(results)
results_df['gap'] = results_df['train_auc'] - results_df['test_auc']
print(results_df.to_string(index=False))

### What do you notice?

- Higher learning rates = bigger train-test gap = overfitting
- Lower learning rates = need more trees to reach same performance

---

## Part 3: Early Stopping

The best way to prevent overfitting: stop when validation performance stops improving.

In [ ]:
# Train with early stopping
lgb_early = lgb.LGBMClassifier(
    n_estimators=1000,  # Set high, let early stopping decide
    max_depth=5,
    learning_rate=0.05,
    random_state=42,
    verbose=-1
)

# Fit with early stopping
lgb_early.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)]
)

print(f"Stopped at {lgb_early.best_iteration_} trees (out of max 1000)")

# Evaluate
early_probs = lgb_early.predict_proba(X_test)[:, 1]
early_auc = roc_auc_score(y_test, early_probs)
print(f"Test AUC: {early_auc:.3f}")

In [ ]:
# Visualize the training process
evals_result = {}
lgb_viz = lgb.LGBMClassifier(
    n_estimators=300, max_depth=5, learning_rate=0.05, random_state=42, verbose=-1
)

lgb_viz.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_names=['train', 'validation'],
    callbacks=[lgb.record_evaluation(evals_result)]
)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(evals_result['train']['binary_logloss'], label='Train')
plt.plot(evals_result['validation']['binary_logloss'], label='Validation')
plt.xlabel('Boosting Round')
plt.ylabel('Log Loss')
plt.title('Training Progress (lower is better)')
plt.legend()
plt.grid(True, alpha=0.3)

# Mark overfitting region
min_val = np.argmin(evals_result['validation']['binary_logloss'])
plt.axvline(x=min_val, color='red', linestyle='--', label=f'Best iteration ({min_val})')
plt.legend()
plt.show()

print(f"\n💡 After iteration {min_val}, validation loss increases while train keeps decreasing.")
print(f"   That's overfitting! Early stopping would stop at {min_val}.")

---

## Part 4: Feature Importance

In [ ]:
# LightGBM feature importance
importance_df = pd.DataFrame({
    'feature': features,
    'importance': lgb_early.feature_importances_
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 5))
plt.barh(importance_df['feature'], importance_df['importance'], color='darkgreen')
plt.xlabel('Importance (Split Count)')
plt.title('LightGBM Feature Importance')
plt.tight_layout()
plt.show()

---

## Part 5: Business Metric Comparison

In [ ]:
# Precision@500 for all models
k = 500
baseline = y_test.mean()

def precision_at_k(y_true, y_proba, k):
    top_k = np.argsort(y_proba)[-k:]
    return y_true.iloc[top_k].mean()

rf_prec = precision_at_k(y_test, rf_model.predict_proba(X_test)[:, 1], k)
lgb_prec = precision_at_k(y_test, lgb_probs, k)
early_prec = precision_at_k(y_test, early_probs, k)

print(f"=== Precision@{k} ===")
print(f"Random baseline:       {baseline:.1%}")
print(f"Random Forest:         {rf_prec:.1%} (lift: {rf_prec/baseline:.1f}x)")
print(f"LightGBM (basic):      {lgb_prec:.1%} (lift: {lgb_prec/baseline:.1f}x)")
print(f"LightGBM (early stop): {early_prec:.1%} (lift: {early_prec/baseline:.1f}x)")

---

## Part 6: Quick Tuning Recipe

This works well in practice—no need for complex grid search.

In [ ]:
# The practical tuning recipe
recipe = """
┌─────────────────────────────────────────────────────────────────────────────┐
│  STEP  │  WHAT TO DO                                                        │
├─────────────────────────────────────────────────────────────────────────────┤
│  1     │  Start with learning_rate=0.05, n_estimators=1000, max_depth=5     │
│  2     │  Use early stopping with 20-50 rounds patience                     │
│  3     │  If overfitting: lower max_depth to 3-4                            │
│  4     │  If underfitting: raise max_depth to 6-8 or num_leaves to 64-127   │
│  5     │  For final model: lower learning_rate to 0.01, increase estimators │
└─────────────────────────────────────────────────────────────────────────────┘

💡 80% of the gains come from:
   - Using early stopping
   - Not having obvious data issues (leakage, wrong features)
"""
print(recipe)

---

## 📝 Final Exercise: Explain It

Your data scientist colleague asks: "Why use LightGBM instead of XGBoost?"

Write a 3-4 sentence response explaining when each is appropriate.

In [ ]:
# Write your response:

colleague_response = """
YOUR RESPONSE HERE

Hint: LightGBM is faster, XGBoost is more battle-tested.
Both give similar results. Pick based on your team's preference.
"""

print(colleague_response)

---

## ✅ Module 4 Complete!

**What you learned:**
- How gradient boosting works (trees fixing previous trees' mistakes)
- The key hyperparameters and their effects
- Why early stopping prevents overfitting
- A practical tuning recipe

In [ ]:
# Summary
print("=== Module 4 Summary ===")
print(f"\nBest Model: LightGBM with early stopping")
print(f"Test AUC: {early_auc:.3f}")
print(f"Precision@500: {early_prec:.1%} ({early_prec/baseline:.1f}x lift)")
print(f"Trees used: {lgb_early.best_iteration_} (vs max 1000)")

**Next:** [Module 5: Feature Engineering →](./module_05_features.ipynb)